In [14]:
assemblyCode = open(r"C:\Users\DripTooHard\Documents\Assignment2Assembly\testCode.txt","r")
assemblyCode = assemblyCode.readlines(10000)


In [15]:
newAssemblyCode = []
for idx,line in enumerate(assemblyCode):
    if line[0] == "#":
        pass
    else:
        newAssemblyCode.append(line)

In [16]:
def decToBin(integer,bitSize):
    return f'{integer:{bitSize}b}'

def extendBits(instruction):
    new_instruction = ""

    #First we add 0's
    for char in instruction:
        if char != "1":
            new_instruction += "0"
        else:
            new_instruction += "1"

    while (len(new_instruction) < 32):
        new_instruction = new_instruction + "0"

    return  new_instruction

#Type A has the pattern: FO:4 R0:4 R1:4 R2:4
def TypeAInstruction(instruction):
    #First we get the decimal values
    instruction = instruction.split()
    R0_dec = instruction[2][1:] #The first char is R, the remaining denotes the index of the register
    R1R2 = instruction[3].split(",")
    R1_dec = R1R2[0][1:]
    R2_dec = R1R2[1][1:]
    list_of_decimal_values = [R0_dec,R1_dec,R2_dec]

    #Now we turn them binary
    function_code_dict = {"ADD":"0000","SUB":"0001","MUL":"0010"}
    function_code = function_code_dict[instruction[1]]

    for idx,value in enumerate(list_of_decimal_values):
        list_of_decimal_values[idx] = decToBin(int(value),4)

    non_32_bit_Instruction = function_code + "".join(list_of_decimal_values)


    return extendBits(non_32_bit_Instruction)

def getBinaryInstruction(funcOpcode,R0,R1,C,lastBitSize = 16):
    return extendBits(funcOpcode + decToBin(int(R0),4) + decToBin(int(R1),4) + decToBin(int(C),lastBitSize) + "0"*(32 - (12 + lastBitSize)))


def assemblyToMachineCode(instruction):
    instruction = instruction.split()

    function_code_dict = {"LOAD":"0100","SAVE":"0101","BNC":"1001","BEQ":"1001","BNQ":"1010","END":"1000","ADDI":"1100","SUBI":"1101","MULI":"1110","ADD":"0000","SUB":"0001","MUL":"0010"}
    function_code = function_code_dict[instruction[1]]

    if instruction[1] == "BNC":
        return getBinaryInstruction(function_code,0,0,instruction[2]) #BNC JUMP = BEQ R0 R0,
    elif instruction[1] == "END":
        return "1" + "0"*31
    elif function_code[0:2] == "01": #Type D
        R0R1 = instruction[2].split(",")
        R0_dec = R0R1[0][1:]
        if function_code[2:4] == "00": #Load
            R1_dec = R0R1[1][1:]
            return getBinaryInstruction(function_code,R0_dec,R1_dec,0)
        else:
            C = R0R1[1]
            return getBinaryInstruction(function_code,0,R0_dec,C)
    elif function_code[0:2] == "00": #Type A
        R0_dec = instruction[2][1:] #The first char is R, the remaining denotes the index of the register
        R1R2 = instruction[3].split(",")
        R1_dec = R1R2[0][1:]
        R2_dec = R1R2[1][1:]
        return getBinaryInstruction(function_code,R0_dec,R1_dec,R2_dec,lastBitSize=4)
    else: #Type I + C - END - BNC
        R0_dec = instruction[2][1:] #The first char is R, the remaining denotes the index of the register
        R1C = instruction[3].split(",")
        R1_dec = R1C[0][1:]
        C = R1C[1]
        return getBinaryInstruction(function_code,R0_dec,R1_dec,C)



In [34]:
#THIS IS THE BLOCK USED TO GET THE MACHINE CODE

assemblyCode = open(r"C:\Users\DripTooHard\Documents\Assignment2Assembly\Assngmentoptimized.txt","r")
assemblyCode = assemblyCode.readlines(10000)

with open(r"C:\Users\DripTooHard\Documents\Assignment2Assembly\machineCodeA2Version2.txt", 'w') as the_file:
    for idx,instruction in enumerate(assemblyCode):
        try:
            if instruction[0] == "#":
                pass
            else:
                print(assemblyToMachineCode(instruction))
                the_file.write(f"\"b{assemblyToMachineCode(instruction)}\".U(32.W),\n")
        except:
            print(idx)

00000011000000000000000000000000
00000100000000000000000000000000
00100101010000100000000000000000
00000101010100110000000000000000
11000110010100000001100100000000
01000111010100000000000000000000
10100000011100000000000010100000
01010000011000000000000000000000
10010000000000000000000111000000
10010000001100000000000011110000
11010111010100000000000000010000
01000111011100000000000000000000
10010000011100000000000010000000
10010001001100000000000100100000
11000111010100000000000000010000
01000111011100000000000000000000
10010000011100000000000010000000
10010000010000000000000101110000
00010111010100100000000000000000
01000111011100000000000000000000
10010000011100000000000010000000
10010001010000000000000110110000
00000111010100100000000000000000
01000111011100000000000000000000
10010000011100000000000010000000
01000111010100000000000000000000
01010000011000000000111111110000
10010001010000000000000111110000
11000100010000000000000000010000
10010000000000000000000000110000
1001000100

In [19]:
TypeIIns = assemblyCode[29]
print(TypeIIns)
result = assemblyToMachineCode(TypeIIns)
result_list = []
for x in range(9):
    lower = (x-1)*4
    upper = x*4
    result_list += [result[lower:upper]]
print(result_list)

27. SAVE R6,255 #We know, that if none of the other conditioned passed, that input_im(x,y) = 255 = output_im(x,y)

['', '0101', '0000', '0110', '0000', '0000', '1111', '1111', '0000']


In [18]:
for idx,line in enumerate(assemblyCode):
    line = line.split()
    if line[1] == "SAVE":
        print(idx)


9
29


In [85]:
for idx,instruction in enumerate(assemblyCode):
    try:
        if instruction[0] == "#":
            pass
        else:
            print(assemblyToMachineCode(instruction))
    except:
        print(idx)


00000011000000000000000000000000
00000100000000000000000000000000
00100101010000100000000000000000
00000101010100110000000000000000
11000110010100000001100011110000
01000111000000000000000001010000
10100000011100000000000010100000
01010110000000000000000000000000
10010000000000000000000111100000
10010000001100000000000011110000
11010111010100000000000000010000
01000111000000000000000001110000
10010000011100000000000010000000
10010001001100000000000100100000
11000111010100000000000000010000
01000111000000000000000001110000
10010000011100000000000010000000
10010000010000000000000101110000
00010111010100100000000000000000
01000111000000000000000001110000
10010000011100000000000010000000
10010000010000000000000111000000
00000111010100100000000000000000
01000111000000000000000001110000
10010000011100000000000010000000
01000111000000000000000001010000
01010110000000000000000001110000
10010010010000000000000111110000
11000100010000000000000000010000
10010000000000000000000000110000
1001001000

In [52]:
a ="0011"
print(a[0:3])

001
